In [18]:
# Steps to install Ollama before running the pipeline.
# Follow the guidelines here - https://ronamosa.io/docs/engineer/AI/Mistral-7B-SageMaker/#jupyterlab-notebook
# Step 1. Run `pip install transformers` on AWS sagemaker terminal
# Step 2. Run `curl -fsSL https://ollama.com/install.sh | sh` on AWS sagemaker terminal
# Step 3. Run `ollama serve` on AWS sagemaker terminal


In [ ]:
# ollama serve is required to serve ollama on a server. After this cell has stopped running, Click on "Kernel -> Restart Kernel" and then move to the next cell
#!ollama serve

In [ ]:
#!ollama run mistral
!ollama run llama3

#!ollama run llama3:70b-text

⠙ ⠹ ⠸ ⠼ ⠴ ⠴ ⠧ ⠇ ⠏ ⠋ ⠋ ⠹ ⠹ ⠸ ⠼ ⠦ ⠦ ⠇ ⠏ ⠋ ⠋ ⠙ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠹ ⠸ ⠴ ⠴ ⠧ ⠇ ⠇ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠦ ⠧ ⠏ ⠏ ⠙ ⠙ ⠸ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠙ ⠹ ⠹ ⠼ ⠴ ⠦ ⠧ ⠇ ⠇ ⠋ ⠙ ⠙ ⠸ ⠸ ⠴ ⠴ ⠧ ⠇ ⠇ ⠋ ⠙ ⠹ ⠸ ⠸ ⠴ ⠴ ⠧ ⠇ ⠇ ⠏ ⠋ ⠙ ⠸ ⠸ ⠼ ⠦ ⠧ ⠇ ⠏ ⠏ ⠋ ⠹ ⠸ ⠸ ⠼ ⠦ ⠧ ⠇ ⠏ ⠏ ⠙ ⠙ ⠸ ⠼ ⠴ ⠴ ⠦ ⠇ ⠇ ⠏ ⠙ ⠹ ⠹ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠏ ⠙ ⠙ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠇ ⠋ ⠋ ⠙ ⠸ ⠼ ⠴ ⠦ ⠧ ⠧ ⠏ ⠋ ⠋ ⠙ ⠸ ⠸ ⠴ ⠴ ⠦ ⠇ ⠇ ⠏ ⠋ ⠙ ⠸ ⠼ ⠴ ⠴ ⠧ ⠇ ⠏ ⠏ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠹ >>> Send a message (/? for help)

In [4]:
"""
import boto3
import pandas as pd
from io import StringIO
import datetime
# Initialize the S3 client
s3_client = boto3.client('s3')

# Define your S3 bucket and file name
bucket_name = 'webscrape-bucket-mle611'
file_name = 'news_data_overall.csv'

# Get the file object from S3
s3_object = s3_client.get_object(Bucket=bucket_name, Key=file_name)
s3_data = s3_object['Body'].read().decode('utf-8')

# Use StringIO to read the CSV data into a pandas DataFrame
input_file = StringIO(s3_data)

# local testing
# input_file = "news_data_overall.csv"

import pandas as pd

# Reading the CSV file
input_csv_path = input_file
df = pd.read_csv(input_csv_path)

# Combining stocks with the same URL
df_combined = df.groupby('URL').agg({
    'Title': 'first',
    'Date': 'first',
    'Source': 'first',
    'Content': 'first',
    'Stock': lambda x: ','.join(set(x))  # Ensuring unique stocks
}).reset_index()

# Saving to a temporary CSV file
temp_csv_path = 'temp_combined_stocks.csv'
df_combined.to_csv(temp_csv_path, index=False)

# Reading the temporary CSV file
df_temp = pd.read_csv(temp_csv_path)

# Creating sentiment analysis strings
stock_mapping = {
    'META': 'Meta',
    'AAPL': 'Apple', 
    'GOOG': 'Google', 
    'AMZN': 'Amazon', 
    'MSFT': 'Microsoft', 
    'NVDA': 'Nvidia', 
    'AMD': 'AMD'
}

def create_sentiment_string(row):
    stocks = list(set(row['Stock'].split(',')))  # Ensuring unique stocks
    stock_names = [stock_mapping[stock] for stock in stocks]
    

    return "Analyze the sentiment of the following text and provide only a decimal number between -1 and 1, where -1 is extremely negative and 1 is extremely positive. Please remember I just want a fraction as output and nothing else - \n\n\n"
    #return f"Create sentiment analysis for the following stocks - {', '.join(stock_names)}. Negative sentiment is indicated from -1 to 0, positive sentiment is indicated from 0 to 1. I only want a number for all the stocks mentioned above separated by commas without spaces, example: Apple:0.51,Microsoft:-0.03"
    

df_temp['SentimentString'] = df_temp.apply(create_sentiment_string, axis=1)

# Displaying the resulting dataframe with sentiment strings
result_df = df_temp[['Content', 'SentimentString', 'URL', 'Stock']]
#print(result_df)

current_date = datetime.datetime.now().strftime('%Y-%m-%d')
# Saving the final result to a new CSV file if needed
final_csv_path = f'combined_stocks_with_sentiment_{current_date}.csv'

#final_csv_path = 'combined_stocks_with_sentiment.csv'
result_df.to_csv(final_csv_path, index=False)

def combine_strings(string1, string2):
    # Combine two strings using an f-string
    return f"{string1}{string2}"
"""

In [ ]:
"""
import csv, codecs, time, json, requests
csv_file_path = final_csv_path
sentiment_file = 'ollamaSentimentOutputs.csv'

# overwrite old file

#fi = codecs.open(sentiment_file, "w", encoding="utf-8")
#fi.close()

# Open the CSV file
with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
    # Create a CSV reader object
    csv_reader = csv.reader(csv_file)
    
    # Skip the header row if your CSV file has one
    header = next(csv_reader)
    
    # Iterate over each row in the CSV file
    for row in csv_reader:
        start_time = time.time()
        # Each row is a list of values
        #print(row[1])
        input_text = row[0]
        sentiment_string = row[1]
        print(sentiment_string)

        url = "http://localhost:11434/api/generate"
        data = {
            "model": "llama3",#"llama3:70b-text",
            "prompt": combine_strings(sentiment_string, input_text)
            #"Analyze the sentiment of the following text and provide only a decimal number between -1 and 1, where -1 is extremely negative and 1 is extremely positive. Please remember I just want a fraction as output and nothing else - \n\n\n" + input_str 
        }

        response = requests.post(url, json=data)
        #print(response)
        print(response.text)

        response_lines = response.text.strip().split('\n')

        # Initialize variables to accumulate the response and check when done
        accumulated_response = ""
        done = False

        # Process each line to extract the sentiment value
        for line in response_lines:
            response_dict = json.loads(line)
            if 'response' in response_dict:
                accumulated_response += response_dict['response']
            if 'done' in response_dict and response_dict['done'] == True:
                done = True
                break
        print(row[2])
        # Print the final sentiment value
        import datetime

        # Get the current date and time
        current_timestamp = datetime.datetime.now()

        # Format the timestamp as a string
        formatted_timestamp = current_timestamp.strftime('%Y-%m-%d %H:%M:%S')

        #print("Formatted timestamp:", formatted_timestamp)
        if done:
            print(accumulated_response.strip())
            fi = codecs.open(sentiment_file, "a+", encoding="utf-8")
            fi.write(str(row[2]) + "," + str(accumulated_response.strip()) + "," + formatted_timestamp + "\n")
            fi.close()
        else:
            print("Incomplete response")
        end_time = time.time()
        print(f"Time taken for this row: {end_time - start_time} seconds")
        
"""

Analyze the sentiment of the following text and provide only a decimal number between -1 and 1, where -1 is extremely negative and 1 is extremely positive. Please remember I just want a fraction as output and nothing else - 



{"model":"llama3","created_at":"2024-06-15T08:21:54.210797931Z","response":"0","done":false}
{"model":"llama3","created_at":"2024-06-15T08:21:54.705337786Z","response":".","done":false}
{"model":"llama3","created_at":"2024-06-15T08:21:55.200257035Z","response":"35","done":false}
{"model":"llama3","created_at":"2024-06-15T08:21:55.713290098Z","response":"","done":true,"done_reason":"stop","context":[128006,882,128007,271,2127,56956,279,27065,315,279,2768,1495,323,3493,1193,264,12395,1396,1990,482,16,323,220,16,11,1405,482,16,374,9193,8389,323,220,16,374,9193,6928,13,5321,6227,358,1120,1390,264,19983,439,2612,323,4400,775,482,15073,7,3905,12019,311,8881,1212,315,1567,11,4442,3338,4939,311,19149,2806,11,7188,8,3296,7639,362,13,56207,3520,323,42107,423,561,258,386,7

In [55]:
import boto3
import pandas as pd
from io import StringIO
import datetime
import codecs
import time
import json
import requests

# Initialize the S3 client
s3_client = boto3.client('s3')

# Define your S3 bucket and file name
bucket_name = 'webscrape-bucket-mle611'
file_name = 'news_data_overall.csv'

# Get the file object from S3
s3_object = s3_client.get_object(Bucket=bucket_name, Key=file_name)

s3_data = s3_object['Body'].read().decode('utf-8')

# Use StringIO to read the CSV data into a pandas DataFrame
input_file = StringIO(s3_data)

# Reading the CSV file
df = pd.read_csv(input_file)

# Combining stocks with the same URL
df_combined = df.groupby('URL').agg({
    'Title': 'first',
    'Date': 'first',
    'Source': 'first',
    'Content': 'first',
    'Stock': lambda x: ','.join(set(x))  # Ensuring unique stocks
}).reset_index()

# Current date for the filename
current_date = datetime.datetime.now().strftime('%Y-%m-%d')
final_csv_path = f'combined_stocks_with_sentiment_{current_date}.csv'
# print(current_date)
# Check if the combined sentiment file already exists and load it
try:
    df_existing_sentiments = pd.read_csv(final_csv_path)
    processed_urls = set(df_existing_sentiments['URL'])
except FileNotFoundError:
    df_existing_sentiments = pd.DataFrame()
    processed_urls = set()

# Filter out rows that have already been processed
df_to_process = df_combined[~df_combined['URL'].isin(processed_urls)]

# Creating sentiment analysis strings
stock_mapping = {
    'META': 'Meta',
    'AAPL': 'Apple', 
    'GOOG': 'Google', 
    'AMZN': 'Amazon', 
    'MSFT': 'Microsoft', 
    'NVDA': 'Nvidia', 
    'AMD': 'AMD'
}

def create_sentiment_string(row):
    stocks = list(set(row['Stock'].split(',')))  # Ensuring unique stocks
    stock_names = [stock_mapping.get(stock, stock) for stock in stocks]
    return "Analyze the sentiment of the following text and provide only a decimal number between -1 and 1, where -1 is extremely negative and 1 is extremely positive. Please remember I just want a fraction as output and nothing else - \n\n\n"

if not df_to_process.empty:
    df_to_process['SentimentString'] = df_to_process.apply(create_sentiment_string, axis=1)

    # Saving to a temporary CSV file
    temp_csv_path = 'temp_combined_stocks.csv'
    df_to_process.to_csv(temp_csv_path, index=False)

    # Reading the temporary CSV file
    df_temp = pd.read_csv(temp_csv_path)

    def combine_strings(string1, string2):
        return f"{string1}{string2}"

    csv_file_path = temp_csv_path
    sentiment_file = 'ollamaSentimentOutputs.csv'

    # Process sentiment analysis for new URLs
    with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)
        
        for row in csv_reader:
            start_time = time.time()
            input_text = row[0]
            sentiment_string = row[1]

            url = "http://localhost:11434/api/generate"
            data = {
                "model": "llama3",
                "prompt": combine_strings(sentiment_string, input_text)
            }

            response = requests.post(url, json=data)
            response_lines = response.text.strip().split('\n')

            accumulated_response = ""
            done = False

            for line in response_lines:
                response_dict = json.loads(line)
                if 'response' in response_dict:
                    accumulated_response += response_dict['response']
                if 'done' in response_dict and response_dict['done'] == True:
                    done = True
                    break

            current_timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            if done:
                fi = codecs.open(sentiment_file, "a+", encoding="utf-8")
                fi.write(f"{row[2]},{accumulated_response.strip()},{current_timestamp}\n")
                fi.close()
            
            end_time = time.time()
            print(f"Time taken for this row: {end_time - start_time} seconds")

    # Combine the existing sentiment data with new sentiment data
    if not df_existing_sentiments.empty:
        result_df = pd.concat([df_existing_sentiments, df_temp[['Content', 'SentimentString', 'URL', 'Stock']]], ignore_index=True)
    else:
        result_df = df_temp[['Content', 'SentimentString', 'URL', 'Stock']]

    # Save the final combined result
    result_df.to_csv(final_csv_path, index=False)
else:
    print("No new URLs to process.")

# Calculate sentiment score accuracy
try:
    manual_sentiment_filename = "manual_sentiment_scores.csv"
    # Get the file object from S3
    s3_object = s3_client.get_object(Bucket="mle-project", Key=manual_sentiment_filename)

    s3_data = s3_object['Body'].read().decode('utf-8')

    # Use StringIO to read the CSV data into a pandas DataFrame
    input_file = StringIO(s3_data)

    # Reading the CSV file
    manual_scores_df = pd.read_csv(input_file)

    #manual_scores_path = 'manual_sentiment_scores.csv'
    #manual_scores_df = pd.read_csv(manual_scores_path)

    # Load the CSV files
    ollama_sentiment = pd.read_csv(sentiment_file, header=None, names=['URL', 'SentimentScore'])

    combined_stocks = pd.read_csv(final_csv_path)

    merged_df = pd.merge(combined_stocks, ollama_sentiment, on='URL', how='left')

    # Split the 'Stock' column into multiple rows
    expanded_df = merged_df.assign(Stock=merged_df['Stock'].str.split(',')).explode('Stock')

    # Calculate the weighted sentiment score per stock and round to three decimal places
    weighted_sentiment = expanded_df.groupby('Stock')['SentimentScore'].mean().reset_index()
    weighted_sentiment['SentimentScore'] = weighted_sentiment['SentimentScore'].round(3)

    # Print the result
    print(weighted_sentiment)

    # Saving the final result to a new CSV file if needed
    target_model_file = f'SA_model_{current_date}.csv'

    weighted_sentiment.to_csv(target_model_file, index=False)

    # Upload to S3
    s3_client.upload_file(target_model_file, 'mle-project', target_model_file)

    print(f"File {target_model_file} uploaded to henry-mleproject-2024")

    # Merge manual scores with calculated scores
    accuracy_df = merged_df.merge(manual_scores_df, on='URL', suffixes=('_calculated', '_manual'))

    # Print column names for debugging
    print("Columns in accuracy_df after merge:", accuracy_df.columns)

    # Calculate accuracy
    accuracy_df['Error'] = (accuracy_df['SentimentScore_calculated'] - accuracy_df['SentimentScore_manual']).abs()
    accuracy = 1 - accuracy_df['Error'].mean()
    print(f"Sentiment analysis accuracy: {accuracy:.2%}")

    # Save the accuracy report if needed
    accuracy_report_path = 'sentiment_accuracy_report.csv'
    accuracy_df.to_csv(accuracy_report_path, index=False)

except FileNotFoundError:
    print("Manual sentiment scores file not found, skipping accuracy calculation.")
print("finished")


No new URLs to process.
  Stock  SentimentScore
0  AAPL           0.465
1   AMD           0.439
2  AMZN           0.547
3  GOOG           0.442
4  META           0.220
5  MSFT           0.421
6  NVDA           0.547
File SA_model_2024-06-21.csv uploaded to henry-mleproject-2024
Columns in accuracy_df after merge: Index(['Content', 'SentimentString', 'URL', 'Stock',
       'SentimentScore_calculated', 'SentimentScore_manual'],
      dtype='object')
Sentiment analysis accuracy: 91.13%
finished


In [ ]:
"""
import pandas as pd

# Load the CSV files
ollama_sentiment = pd.read_csv(sentiment_file, header=None, names=['URL', 'SentimentScore'])

combined_stocks = pd.read_csv(final_csv_path)

merged_df = pd.merge(combined_stocks, ollama_sentiment, on='URL', how='left')

# Split the 'Stock' column into multiple rows
expanded_df = merged_df.assign(Stock=merged_df['Stock'].str.split(',')).explode('Stock')

# Calculate the weighted sentiment score per stock and round to three decimal places
weighted_sentiment = expanded_df.groupby('Stock')['SentimentScore'].mean().reset_index()
weighted_sentiment['SentimentScore'] = weighted_sentiment['SentimentScore'].round(3)

# Print the result
print(weighted_sentiment)

# Saving the final result to a new CSV file if needed
target_model_file = f'SA_model_{current_date}.csv'

#target_model_file = "SA_model.csv"
weighted_sentiment.to_csv(target_model_file, index=False)

import boto3

# Initialize the S3 client
s3_client = boto3.client('s3')

bucket_name = 'henry-mleproject-2024'
file_name = target_model_file

s3_client.upload_file(target_model_file, bucket_name, file_name)

print(f"File {file_name} uploaded to {bucket_name}")
"""